In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
%matplotlib inline

In [3]:
## 주피터노트북 cell 너비 조정

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [10]:
from konlpy.tag import Twitter
m = Twitter()

In [11]:
import codecs
with codecs.open("ratings_train.txt", encoding='utf-8') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]   # header 제외

## 1. 형태소 분석

In [8]:
data[0][1]

'아 더빙.. 진짜 짜증나네요 목소리'

In [28]:
import konlpy.tag as tag

In [7]:
## 품사 태깅
m.pos(data[0][1])

[('아', 'IC'),
 ('더빙', 'NNG'),
 ('.', 'SF'),
 ('.', 'SY'),
 ('진짜', 'MAG'),
 ('짜증', 'NNG'),
 ('나', 'NP'),
 ('네요', 'VCP+EC'),
 ('목소리', 'NNG')]

In [9]:
## 명사만 추출

m.nouns(data[0][1])

['더빙', '짜증', '나', '목소리']

In [29]:
m.pos('아기다리고기다리던소풍')

[('아기', 'NNG'),
 ('다리', 'VV'),
 ('고', 'EC'),
 ('기다리', 'VV'),
 ('던', 'ETM'),
 ('소풍', 'NNG')]

In [31]:
from konlpy.tag import Kkma
k = Kkma()

In [32]:
k.pos('아기다리고기다리던소풍')

[('아', 'VV'),
 ('아', 'ECS'),
 ('기다리', 'VV'),
 ('고', 'ECE'),
 ('기다리', 'VV'),
 ('더', 'EPT'),
 ('ㄴ', 'ETD'),
 ('소풍', 'NNG')]

## 2. Word2Vec

- 단어를 벡터화하는 방법론
    - 단어 임베딩(Word Embedding) : 텍스트를 구성하는 하나의 단어를 수치화하는 방법의 일종.
- word2vec은 CBOW 방식과 Skip-Gram 방식의 단어 임베딩을 구현한 C++ 라이브러리로 구글에 있던 Mikolov 등이 개발하였다.
    - CBOW (Continuos Bag Of Words ) : 연속된 여러 단어를 보고 그 다음에 올 단어를 예측하는 방법
    - Skip-Gram : 하나의 단어를 보고 그 다음에 올 문맥을 예측하는 방법
- 결국, 앞 뒤간의 관계성을 파악하는 알고리즘이므로 다양한 분야에 활용가능하다.
    - sequential mining, 추천시스템 등
    
<img src="https://datascienceschool.net/upfiles/3cdbdbfe1c8a4742aaf2e0c40917948f.png" alt="Drawing" style="width: 10px;" align = "left"/>
<img src="https://datascienceschool.net/upfiles/8f8ebfa0ebb34eb584d24d59fe60a12d.png" alt="Drawing" style="width: 10px;" align = "right"/>

> ### 임베딩의 장점
- 벡터공간에서 텍스트들간의 관계를 파악할 수 있고 수학적 계산을 할 수 있다.


<img src="https://www.tensorflow.org/versions/r0.11/images/linear-relationships.png" alt="Drawing" style="width: 10px;" align = "left"/>

In [12]:
## 명사만 추출

from tqdm import tqdm

for _, row in tqdm(enumerate(data)):
    row.append(m.nouns(row[1]))

150000it [00:25, 5777.82it/s]


In [14]:
from gensim.models.word2vec import Word2Vec

In [19]:
## word2vec 모델링
model = Word2Vec([row[-1] for _, row in enumerate(data)])

In [26]:
## 코사인 거리가 가장 가까운 순서대로 
# 배우

model.most_similar('정우성')

[('이연희', 0.9758511185646057),
 ('박중훈', 0.9746931195259094),
 ('강동원', 0.9729968309402466),
 ('김수로', 0.9711499810218811),
 ('전지현', 0.9706459641456604),
 ('송혜교', 0.9694119095802307),
 ('이준기', 0.9674740433692932),
 ('권상우', 0.9669971466064453),
 ('이승기', 0.9667418003082275),
 ('이범수', 0.9663347601890564)]

In [27]:
## 코사인 거리가 가장 가까운 순서대로 
# 감독

model.most_similar('김기덕')

[('홍상수', 0.9527831673622131),
 ('천재', 0.9273989796638489),
 ('장진', 0.9122887253761292),
 ('임권택', 0.9025713801383972),
 ('연줄', 0.8987452983856201),
 ('최양일', 0.8937935829162598),
 ('의심', 0.8928626179695129),
 ('능력', 0.8905345797538757),
 ('재능', 0.8829715251922607),
 ('이장호', 0.8772603869438171)]